In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import requests
import os
import time

In [2]:
stock_list = pd.read_csv('list_of_stocks.csv')
stock_list.head()

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,CUSIP,ISIN,SEDOL,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,AAPL,APPLE INC,Information Technology,Equity,"1,875,566,056.60",4.85,"1,875,566,056.60","14,040,770.00",37833100,US0378331005,2046251,133.58,United States,NASDAQ,USD,1,USD,--
1,MSFT,MICROSOFT CORP,Information Technology,Equity,"1,709,995,853.92",4.42,"1,709,995,853.92","6,717,457.00",594918104,US5949181045,2588173,254.56,United States,NASDAQ,USD,1,USD,--
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"1,319,116,860.50",3.41,"1,319,116,860.50","381,413.00",23135106,US0231351067,2000019,"3,458.50",United States,NASDAQ,USD,1,USD,--
3,FB,FACEBOOK CLASS A INC,Communication,Equity,"659,282,587.10",1.71,"659,282,587.10","2,146,801.00",30303M102,US30303M1027,B7TL820,307.1,United States,NASDAQ,USD,1,USD,--
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,"633,933,024.00",1.64,"633,933,024.00","268,725.00",02079K305,US02079K3059,BYVY8G0,"2,359.04",United States,NASDAQ,USD,1,USD,--


In [3]:
# this code pulls in the .xlsx file
url='https://stockrow.com/api/companies/AAPL/financials.xlsx?dimension=T&section=Income%20Statement&sort=desc'
response = requests.get(url)
with open(os.path.join("Income", "AAPL_inc.xlsx"), 'wb') as f:
    f.write(response.content)

In [10]:
ticker_list = stock_list['Ticker'][:10]

for ticker in ticker_list:
    url=f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=T&section=Income%20Statement&sort=desc'
    response = requests.get(url)
    with open(os.path.join("Income", f"{ticker}_inc.xlsx"), 'wb') as f:
        f.write(response.content)
    time.sleep(0.25)

In [44]:
#read in the DataFrame, transposed
AAPL_df = pd.read_excel('./Income/AAPL_inc.xlsx').T
# change the first row to the columns
AAPL_df.columns = AAPL_df.iloc[0]
#remove the first row
AAPL_df = AAPL_df.iloc[1:]

AAPL_df.head()

Unnamed: 0,Revenue,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expenses,Operating Income,Interest Expense (Operating),Non-operating Income/Expense,Non-operating Interest Expenses,EBT,...,Free Cash Flow Margin,EBITDA,EBIT,"EPS (Diluted, from Cont. Ops)","EPS (Basic, Consolidated)","EPS (Basic, from Continuous Ops)",Income from Continuous Operations,Consolidated Net Income/Loss,EBITDA Margin,Operating Cash Flow Margin
2020-12-31 00:00:00,2.93971e+11,1.80236e+11,1.13735e+11,1.9464e+10,2.035e+10,7.4253e+10,2.726e+09,-7.46e+08,3.465e+09,7.4752e+10,...,0.2729,8.4827e+10,7.4253e+10,3.7076,3.7474,3.7474,6.393e+10,6.393e+10,0.2886,0.3025
2020-09-30 00:00:00,2.7415e+11,1.70143e+11,1.04007e+11,1.8752e+10,1.9916e+10,6.6288e+10,2.873e+09,-6.8e+07,3.763e+09,6.7091e+10,...,0.2676,7.6395e+10,6.6288e+10,3.2753,3.3085,3.3085,5.7411e+10,5.7411e+10,0.2787,0.2943
2020-06-30 00:00:00,2.73524e+11,1.69798e+11,1.03726e+11,1.7884e+10,1.9558e+10,6.7138e+10,NaN,5.31e+08,NaN,6.8317e+10,...,0.2622,7.7817e+10,6.7138e+10,3.2976,3.3262,3.3262,5.8424e+10,5.8424e+10,0.2845,0.2925
2020-03-31 00:00:00,2.67918e+11,1.66013e+11,1.01905e+11,1.7383e+10,1.9153e+10,6.5591e+10,NaN,6.62e+08,NaN,6.7091e+10,...,0.2487,7.7083e+10,6.5591e+10,3.1973,3.2233,3.2233,5.7215e+10,5.7215e+10,0.2877,0.2813
2019-12-31 00:00:00,2.67586e+11,1.66276e+11,1.0131e+11,1.6766e+10,1.8659e+10,6.6153e+10,NaN,1.348e+09,NaN,6.7749e+10,...,0.2391,7.7853e+10,6.6153e+10,3.1737,3.1967,3.1967,5.7527e+10,5.7527e+10,0.2909,0.2736


In [40]:
AAPL_df

Unnamed: 0,Revenue,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expenses,Operating Income,Interest Expense (Operating),Non-operating Income/Expense,Non-operating Interest Expenses,EBT,...,Free Cash Flow Margin,EBITDA,EBIT,"EPS (Diluted, from Cont. Ops)","EPS (Basic, Consolidated)","EPS (Basic, from Continuous Ops)",Income from Continuous Operations,Consolidated Net Income/Loss,EBITDA Margin,Operating Cash Flow Margin
Unnamed: 0,Revenue,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expenses,Operating Income,Interest Expense (Operating),Non-operating Income/Expense,Non-operating Interest Expenses,EBT,...,Free Cash Flow Margin,EBITDA,EBIT,"EPS (Diluted, from Cont. Ops)","EPS (Basic, Consolidated)","EPS (Basic, from Continuous Ops)",Income from Continuous Operations,Consolidated Net Income/Loss,EBITDA Margin,Operating Cash Flow Margin
2020-12-31 00:00:00,2.93971e+11,1.80236e+11,1.13735e+11,1.9464e+10,2.035e+10,7.4253e+10,2.726e+09,-7.46e+08,3.465e+09,7.4752e+10,...,0.2729,8.4827e+10,7.4253e+10,3.7076,3.7474,3.7474,6.393e+10,6.393e+10,0.2886,0.3025
2020-09-30 00:00:00,2.7415e+11,1.70143e+11,1.04007e+11,1.8752e+10,1.9916e+10,6.6288e+10,2.873e+09,-6.8e+07,3.763e+09,6.7091e+10,...,0.2676,7.6395e+10,6.6288e+10,3.2753,3.3085,3.3085,5.7411e+10,5.7411e+10,0.2787,0.2943
2020-06-30 00:00:00,2.73524e+11,1.69798e+11,1.03726e+11,1.7884e+10,1.9558e+10,6.7138e+10,NaN,5.31e+08,NaN,6.8317e+10,...,0.2622,7.7817e+10,6.7138e+10,3.2976,3.3262,3.3262,5.8424e+10,5.8424e+10,0.2845,0.2925
2020-03-31 00:00:00,2.67918e+11,1.66013e+11,1.01905e+11,1.7383e+10,1.9153e+10,6.5591e+10,NaN,6.62e+08,NaN,6.7091e+10,...,0.2487,7.7083e+10,6.5591e+10,3.1973,3.2233,3.2233,5.7215e+10,5.7215e+10,0.2877,0.2813
2019-12-31 00:00:00,2.67586e+11,1.66276e+11,1.0131e+11,1.6766e+10,1.8659e+10,6.6153e+10,NaN,1.348e+09,NaN,6.7749e+10,...,0.2391,7.7853e+10,6.6153e+10,3.1737,3.1967,3.1967,5.7527e+10,5.7527e+10,0.2909,0.2736
2019-09-30 00:00:00,2.60174e+11,1.61782e+11,9.8392e+10,1.6217e+10,1.8245e+10,6.393e+10,NaN,1.807e+09,NaN,6.5737e+10,...,0.2264,7.6477e+10,6.393e+10,2.9715,2.9915,2.9915,5.5256e+10,5.5256e+10,0.2939,0.2667
2019-06-30 00:00:00,2.59034e+11,1.60871e+11,9.8163e+10,1.5857e+10,1.7883e+10,6.4423e+10,NaN,1.608e+09,NaN,6.6031e+10,...,0.2249,7.6545e+10,6.4423e+10,2.9439,2.9664,2.9664,5.5695e+10,5.5695e+10,0.2955,0.2664
2019-03-31 00:00:00,2.5849e+11,1.60133e+11,9.8357e+10,1.5301e+10,1.7565e+10,6.5491e+10,NaN,1.913e+09,NaN,6.7404e+10,...,0.2315,7.7345e+10,6.5491e+10,2.9827,3.0069,3.0069,5.717e+10,5.717e+10,0.2992,0.278
2018-12-31 00:00:00,2.61612e+11,1.61654e+11,9.9958e+10,1.4731e+10,1.7257e+10,6.797e+10,NaN,1.809e+09,NaN,6.9779e+10,...,0.2369,7.9523e+10,6.797e+10,3.0496,3.0762,3.0762,5.9431e+10,5.9431e+10,0.304,0.2899
